In [7]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch import nn
from torchvision import datasets, transforms, models
from PIL import Image
import os
import random
import time
import csv
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [13]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [8]:
def im_convert(tensor):
  image = tensor.cpu().clone().detach().numpy()
  image = image.transpose(1, 2, 0)
  image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
  image = image.clip(0, 1)
  return image

#Directory definitions

train_directory = r'../data/imgs/train'
test_directory = r'../data/imgs/test'
main_directory = r'../data'
device = torch.device("cuda:0")

In [9]:
def train_vgg_model(device):
    model = models.vgg16(pretrained=True)
    # Freezing the feature parameters
    for param in model.features.parameters():
      param.requires_grad = False

    # Defining the loss, optimizer and learning rate
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    # changing the last layer of VGG16 to have a size of 10
    n_inputs = model.classifier[6].in_features
    last_layer = nn.Linear(n_inputs, 10)
    model.classifier[6] = last_layer
    model.to(device)
    # load previously saved weights
    #model.load_state_dict(torch.load(os.path.join(main_directory, '20220302_201239_weights.h5')))
    # defining the softmax for converting the output to probabilities
    module_softmax = nn.Softmax(dim=1)
    #print(model)
    
    epochs = 20
    running_loss_history = []
    running_corrects_history = []
    val_running_loss_history = []
    val_running_corrects_history = []

    for e in range(epochs):

        running_loss = 0.0
        running_corrects = 0.0
        val_running_loss = 0.0
        val_running_corrects = 0.0

        for inputs, labels in training_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item()
            running_corrects += torch.sum(preds == labels.data)

        else:
            epoch_loss = running_loss / len(training_loader.dataset)
            epoch_acc = running_corrects.float() / len(training_loader.dataset)
            running_loss_history.append(epoch_loss)
            running_corrects_history.append(epoch_acc)
            print('epoch :', (e + 1))
            print('training loss: {:.4f}, acc {:.4f}, num of corrects {} '.format(epoch_loss, epoch_acc.item(), running_corrects))

    # saving the weights
    file_date = datetime.now().strftime("%Y%m%d_%H%M%S")
    torch.save(model.state_dict(), os.path.join(main_directory, file_date + '_weights.h5'))
    

In [6]:
# transformation to resize, add color jitter and normalize the image and convert it to a tensor
transform_train = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                      ])

# transformation to resize and normalize the test images and convert them to a tensor
transform_test = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                     ])

# using ImageFolder command to read the images, classify them and transform them
training_dataset = datasets.ImageFolder(train_directory, transform=transform_train)

training_loader = torch.utils.data.DataLoader(training_dataset, batch_size = len(training_dataset), shuffle=True)



classes = ('c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9')
action = ('normal driving', 'texting - right', 'phone - right', 'texting - left', 'phone - left', 'radio',
          'drinking', 'behind', 'hair and makeup', 'passenger')

dataiter = iter(training_loader)
images, labels = dataiter.next()

print(len(images))
print(len(labels))

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

count = 1

for train_index, val_index in skf.split(images, labels):
    
    #this runs the action of splitting into training/validation sets n_splits (in this caes, 10) times
    #in here, we will also train any model we want (hence resulting in training the model n_splits times)
    
    print("Iteration number: " + str(count))
    print(train_index)
    print(val_index)
    X_train, X_val = images[train_index], images[val_index]
    Y_train, Y_val = labels[train_index], labels[val_index]
    
    #train whatever network on X_train
    train_vgg_model(device)
    count += 1
    


22424
22424
Iteration number: 1
[    0     1     2 ... 22421 22422 22423]
[    6    22    23 ... 22404 22408 22411]


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
!